<a href="https://colab.research.google.com/github/sethi-rasna/Wine_Os/blob/main/MACHINE_LEARNING_final_group_project_MACHINE_LEARNING_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS Red_White_Wine;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE Red_White_Wine;'

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 10.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package cron.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../0-cron_3.0pl1-128.1ubuntu1.2_amd64.deb ...
Unpacking cron (3.0pl1-128.1ubuntu1.2) ...
Selecting previously unselected package logrotate.
Preparing to unpack .../1-logrotate_3.11.0-0.1ubuntu1_amd64.deb ...
Unpacking logrotate (3.11.0-0.1ubuntu1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../2-netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Selecting previously unselected

In [2]:
%env Red_White_Wine_DATABASE_NAME=Red_White_Wine
%env Red_White_Wine_DATABASE_HOST=localhost
%env Red_White_Wine_DATABASE_PORT=5432
%env Red_White_Wine_DATABASE_USER=postgres
%env Red_White_Wine_DATABASE_PASS=!Hebob77

env: Red_White_Wine_DATABASE_NAME=Red_White_Wine
env: Red_White_Wine_DATABASE_HOST=localhost
env: Red_White_Wine_DATABASE_PORT=5432
env: Red_White_Wine_DATABASE_USER=postgres
env: Red_White_Wine_DATABASE_PASS=!Hebob77


In [3]:
from google.colab import files
upload =  files.upload()

Saving red_white.csv to red_white.csv


**Import Dependencies**

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Import our input dataset
application_df = pd.read_csv('red_white.csv')
application_df.head()

,wine_type,winetype_numeric,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,fixed acidity.1,sulphates 1=low 2=high,alcohol 1=low 2=high,fixed acidity 1=low 2=high,healthiness,quality
0,white,2,3.8,0.310,0.02,11.1,0.036,20.0,114.0,0.99248,3.75,0.44,12.4,3.8,1,2,1,1,2
1,white,2,3.9,0.225,0.40,4.2,0.030,29.0,118.0,0.98900,3.57,0.36,12.8,3.9,1,2,1,1,2
2,white,2,4.4,0.460,0.10,2.8,0.024,31.0,111.0,0.98816,3.48,0.34,13.1,4.4,1,2,1,1,2
3,white,2,4.8,0.225,0.38,1.2,0.074,47.0,130.0,0.99132,3.31,0.40,10.3,4.8,1,2,1,1,2
4,white,2,4.8,0.170,0.28,2.9,0.030,22.0,111.0,0.99020,3.38,0.34,11.3,4.8,1,2,1,1,2


In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
%matplotlib inline

**Data Preprocessing**

In [6]:
#Drop the non-beneficial ID columns, 'volatile acidity','citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide'.
application_df = application_df.drop(['wine_type','volatile acidity','citric acid', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide'],1)
application_df.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,winetype_numeric,fixed acidity,residual sugar,density,pH,sulphates,alcohol,fixed acidity.1,sulphates 1=low 2=high,alcohol 1=low 2=high,fixed acidity 1=low 2=high,healthiness,quality
0,2,3.8,11.1,0.99248,3.75,0.44,12.4,3.8,1,2,1,1,2
1,2,3.9,4.2,0.98900,3.57,0.36,12.8,3.9,1,2,1,1,2
2,2,4.4,2.8,0.98816,3.48,0.34,13.1,4.4,1,2,1,1,2
3,2,4.8,1.2,0.99132,3.31,0.40,10.3,4.8,1,2,1,1,2
4,2,4.8,2.9,0.99020,3.38,0.34,11.3,4.8,1,2,1,1,2
5,2,4.8,1.1,0.99246,3.32,0.36,13.5,4.8,1,2,1,1,2
6,1,5.0,1.4,0.99380,3.75,0.48,10.5,5.0,1,2,1,1,2
7,2,5.0,1.5,0.99170,3.48,0.44,10.7,5.0,1,2,1,1,2
8,2,5.0,1.5,0.99060,3.48,0.39,10.8,5.0,1,2,1,1,2
9,2,5.0,4.5,0.98956,3.45,0.31,12.6,5.0,1,2,1,1,2


In [7]:
# Determine the number of unique values in each column.
application_df.nunique()

winetype_numeric                2
fixed acidity                 102
residual sugar                254
density                       752
pH                             98
sulphates                     108
alcohol                        84
fixed acidity.1               102
sulphates 1=low 2=high          2
alcohol 1=low 2=high            2
fixed acidity 1=low 2=high      2
healthiness                     2
quality                         2
dtype: int64

In [8]:
# Look at winetype_numeric value counts for binning
application_count = application_df.winetype_numeric.value_counts()
application_count


2    1470
1    1470
Name: winetype_numeric, dtype: int64

In [9]:
# Generate our categorical variable lists 
#application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()
#application_df[application_cat].nunique()

In [10]:
# define "healthy" column with metrics scores based on sulphate content. IF low then 1= HEALTHY If high then 2= NOT HEALTHY 

y = application_df[['healthiness','quality']].values
X = application_df.drop(['healthiness','quality'],1).values 

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42, stratify=y)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



**Compile, Train, and Evaluate the Model**

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30

nn = Sequential()

# First hidden layer
nn.add(Dense(units=hidden_nodes_layer1,input_dim=number_input_features,activation='relu'))

# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2,activation='relu'))

# Output layer
nn.add(Dense(units=2,activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                960       
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 2)                 62        
                                                                 
Total params: 3,452
Trainable params: 3,452
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
69/69 [==============================] - 1s 2ms/step - loss: -1.5968 - accuracy: 0.3492
Epoch 2/100
69/69 [==============================] - 0s 2ms/step - loss: -15.3816 - accuracy: 0.5215
Epoch 3/100
69/69 [==============================] - 0s 2ms/step - loss: -80.0536 - accuracy: 0.6930
Epoch 4/100
69/69 [==============================] - 0s 2ms/step - loss: -260.9406 - accuracy: 0.8082
Epoch 5/100
69/69 [==============================] - 0s 2ms/step - loss: -627.0150 - accuracy: 0.8222
Epoch 6/100
69/69 [==============================] - 0s 2ms/step - loss: -1241.3762 - accuracy: 0.8063
Epoch 7/100
69/69 [==============================] - 0s 2ms/step - loss: -2165.1682 - accuracy: 0.8063
Epoch 8/100
69/69 [==============================] - 0s 2ms/step - loss: -3449.0420 - accuracy: 0.8063
Epoch 9/100
69/69 [==============================] - 0s 2ms/step - loss: -5154.5010 - accuracy: 0.8063
Epoch 10/100
69/69 [==============================] - 0s 2ms/step - loss: -7318.34

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

23/23 - 0s - loss: -4.2610e+06 - accuracy: 0.8054 - 370ms/epoch - 16ms/step
Loss: -4261005.5, Accuracy: 0.8054421544075012


In [16]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/Optimization1/',exist_ok=True)
checkpoint_path = 'checkpoints/Optimization1/weights.{epoch:02d}.hdf5'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=100)

# Compile the model
nn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
69/69 [==============================] - 1s 5ms/step - loss: -4345513.5000 - accuracy: 0.8063
Epoch 2/100
30/69 [============>.................] - ETA: 0s - loss: -4272081.0000 - accuracy: 0.7937
Epoch 2: saving model to checkpoints/Optimization1/weights.02.hdf5
69/69 [==============================] - 0s 4ms/step - loss: -4433203.5000 - accuracy: 0.8063
Epoch 3/100
61/69 [=========================>....] - ETA: 0s - loss: -4507413.0000 - accuracy: 0.8064
Epoch 3: saving model to checkpoints/Optimization1/weights.03.hdf5
69/69 [==============================] - 0s 5ms/step - loss: -4522305.0000 - accuracy: 0.8063
Epoch 4/100
69/69 [==============================] - 0s 4ms/step - loss: -4613027.5000 - accuracy: 0.8063
Epoch 5/100
19/69 [=======>......................] - ETA: 0s - loss: -4506849.0000 - accuracy: 0.7780
Epoch 5: saving model to checkpoints/Optimization1/weights.05.hdf5
69/69 [==============================] - 0s 5ms/step - loss: -4705439.0000 - accuracy: 0.8063

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

23/23 - 0s - loss: -2.2430e+07 - accuracy: 0.8054 - 162ms/epoch - 7ms/step
Loss: -22429520.0, Accuracy: 0.8054421544075012


In [18]:
# Export our model to HDF5 file
nn.save('groupprojectMachineLearning_model')
